In [1]:
import awkward as ak
import numpy as np
import hist as hs
from coffea import processor, hist as chs
from coffea.nanoevents.methods import vector, candidate
from numba import jit

from coffea.nanoevents import BaseSchema
import mplhep as hep
import matplotlib.pyplot as plt
plt.style.use(hep.style.CMS)

/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
class LLP_ntuple_processor(processor.ProcessorABC):
    """
    This class is used to process the ntuples created by the LLP ntuple producer.
    """

    def __init__(self):
        # create a dictionary for storing dimensions of different parts of the detector
        self.detector_dimensions = {
            'csc': {},
            'dt': {},
            'cms': {},
        }

        # fill detector_dimensions with the dimensions of the different parts of the detector
        s = 1e2  # scale factor
        self.detector_dimensions['csc']['zmin'] = s * 5.5
        self.detector_dimensions['csc']['zmax'] = s * 10.
        self.detector_dimensions['csc']['rmin'] = s * 0.
        self.detector_dimensions['csc']['rmax'] = s * 7.
        self.detector_dimensions['dt']['zmin'] = s * 0.
        self.detector_dimensions['dt']['zmax'] = s * 6.5
        self.detector_dimensions['dt']['rmin'] = s * 4.
        self.detector_dimensions['dt']['rmax'] = s * 7.5

        self.detector_dimensions['cms']['zmin'] = s * 0.
        self.detector_dimensions['cms']['zmax'] = s * 16.
        self.detector_dimensions['cms']['rmin'] = s * 0.
        self.detector_dimensions['cms']['rmax'] = s * 12.

    def llp_cut(self, events, pid):
        """
        this function will filter events that have gLLPs
        :param events:
        :return:
        """
        cut = events.gParticleId == pid
        return events[ak.any(cut, axis = -1)]


    '''def muon_pt_cut(self, events, pid):
        
        #gets the indexes of the mothers of LLPs
        llp_mother_index = ak.flatten(events.gParticleMotherIndex[events.gParticleId == pid], axis=None)

        #cuts for muons that share the same mothers as LLPs
        cut = events.gParticlePt[(events.gParticleMotherIndex == llp_mother_index[:,None]) 
                            & (abs(events.gParticleId) == 13)] >= 9

        return events[ak.any(cut, axis = -1)]

    def muon_eta_cut(self, events, pid):
        
        llp_mother_index = ak.flatten(events.gParticleMotherIndex[events.gParticleId == pid], axis=None)
        cut = abs(events.gParticleEta[(events.gParticleMotherIndex == llp_mother_index[:,None]) & 
                                 (abs(events.gParticleId) == 13)]) <= 1.5

        return events[ak.any(cut, axis = -1)]'''

    '''def muon_pt_cut(self, events, pid):
        
        cut = abs(events.muonPt) >= 9

        return events[ak.any(cut, axis = -1)]

    def muon_eta_cut(self, events, pid):
        
        cut = abs(events.muonEta) < 1.5

        return events[ak.any(cut, axis = -1)]'''

    def muon_cut(self, events, pid):
        """
        this function will filter events that have gLLPs
        :param events:
        :return:
        """
        #cut = ((abs(events.muonPt) >= 9) & (abs(events.muonEta) < 1.5))
        cut = ((abs(events.muonEta) < 1.5))
        return events[ak.any(cut, axis = -1)]


    '''def muon_cut(self, events, pid):
        
        llp_mother_index = ak.flatten(events.gParticleMotherIndex[events.gParticleId == pid], axis=None)

        #cuts for muons that share the same mothers as LLPs
        cutpt = events.gParticlePt[(events.gParticleMotherIndex == llp_mother_index[:,None]) 
                            & (abs(events.gParticleId) == 13)] >= 9

        cuteta = abs(events.gParticleEta[(events.gParticleMotherIndex == llp_mother_index[:,None]) & 
                                 (abs(events.gParticleId) == 13)]) <= 1.5

        return events[ak.any((cutpt & cuteta), axis = -1)]'''


    def csc_cut(self, events, pid):
        """
        this function will filter events that have gLLPs within the CSC
        :param events:
        :return:
        """
        # we need the decay vertices of the llps but events doesn't have that information. but we do have access to
        # the id of the mother particles (gParticleMotherId) as well as production vertices of all particles (
        # gParticleProdVertexX, gParticleProdVertexY, gParticleProdVertexZ) Make a mask of the particles whose mother
        # is the llp
        mother_llp_mask = events.gParticleMotherId == pid
        # However, each llp will have multiple products, so events.gParticleProdVertexX[mother_llp_mask] will have
        # duplicates To remove the duplicates we can use a trick that uses ak.argmax to find the index of the first
        # occurrence of each truth value Make an awkward array of the indices of the first occurrence of each truth
        # value within the second level of the mask
        llp_daughter_index = ak.argmax(mother_llp_mask, axis=1, keepdims=True)
        # Now we can use llp_daughter_index to get the indexes of the llps themselves using gParticleMotherIndex
        llp_index = events.gParticleMotherIndex[llp_daughter_index]

        def R(x, y):
            return np.sqrt(x ** 2 + y ** 2)

        dims = self.detector_dimensions['csc']
        cut = (
                (abs(events.gParticleEta) < 2.4) &
                (abs(events.gParticleProdVertexZ) > dims['zmin']) & (abs(events.gParticleProdVertexZ) < dims['zmax']) &
                (R(events.gParticleProdVertexX, events.gParticleProdVertexX) < dims['rmax'])
        )
        return events[ak.any(cut, axis = -1)]

    def timetotal_cut(self, events, pid):
        cut = ((events.cscRechitClusterTimeTotal >= -5) & 
               (events.cscRechitClusterTimeTotal <= 12.5))
        
        return events[ak.any(cut, axis = -1)]

    def ME1112_veto(self, events, pid):
        cut = (
                (events.cscRechitClusterNRechitChamberPlus11 <= 0) &
                (events.cscRechitClusterNRechitChamberPlus12 <= 0) &
                (events.cscRechitClusterNRechitChamberMinus11 <= 0) &
                (events.cscRechitClusterNRechitChamberMinus12 <= 0)
        )
        return events[ak.any(cut, axis = -1)]

    def eta_cut(self, events, pid): 
        cut = (abs(events.gParticleEta) < 2)
        return events[ak.any(cut, axis = -1)]

    def timeSpread_cut(self, events, pid):
        cut = (events.cscRechitClusterTimeSpread <= 20)
        return events[ak.any(cut, axis = -1)]


    def process(self, events):
        """
        This function is used to do addition basically.
        :param events:
        :return:
        """

        dataset = events.metadata['dataset']
        sumw = ak.sum(events.genWeight)

        out = {
            dataset: {
                "entries": len(events),
                "sumw": sumw,
            }
        }

        # simple if statement that assigns a variable 'pid' to 9900015 or 1000023 if any of the events has one of
        # those particles
        if ak.any(events.gParticleId == 9900015): pid = 9900015
        if ak.any(events.gParticleId == 1000023): pid = 1000023

        events_with_llp = self.llp_cut(events, pid)
        
        #muon_pt_cut = self.muon_pt_cut(events_with_llp, pid)
        #muon_eta_cut = self.muon_eta_cut(muon_pt_cut, pid)

        muon_cut = self.muon_cut(events_with_llp, pid)

        
        #csc_acc_cut = self.csc_cut(muon_eta_cut, pid)
        csc_acc_cut = self.csc_cut(muon_cut, pid)
        csc_cls_cut = csc_acc_cut[csc_acc_cut.nCscRechitClusters >= 1]
        timetotal_cut = self.timetotal_cut(csc_cls_cut, pid)
        ME1112_veto = self.ME1112_veto(timetotal_cut, pid)
        eta_cut = self.eta_cut(ME1112_veto, pid)
        timeSpread_cut = self.timeSpread_cut(eta_cut, pid)
  
        out[dataset]['events_with_llp'] = len(events_with_llp)
        #out[dataset]['muon_pt_cut'] = len(muon_pt_cut)
        #out[dataset]['muon_eta_cut'] = len(muon_eta_cut)
        out[dataset]['muon_cut'] = len(muon_cut)
        out[dataset]['csc_acc_cut'] = len(csc_acc_cut)
        out[dataset]['csc_cls_cut'] = len(csc_cls_cut)
        out[dataset]['timetotal_cut'] = len(timetotal_cut)
        out[dataset]['ME1112_veto'] = len(ME1112_veto)
        out[dataset]['eta_cut'] = len(eta_cut)
        out[dataset]['timeSpread_cut'] = len(timeSpread_cut)

        print(events)
        print(csc_acc_cut)



        return out

    def postprocess(self, accumulator):
        return accumulator

In [6]:
# digging up
def rootAdds(directory):
    my_file = open(directory, "r")
    data = my_file.read().strip()
    data_into_list = data.split("\n")
    my_file.close()
    return data_into_list

fileset = {}
#fileset['hnl'] = rootAdds('/LLPs/notebooks/Bplus/rootAdds/BToHNL_MuonAndHNLGenFilter_mHNL1p0_ctau1000.txt') # for when working in vs
fileset['hnl'] = rootAdds('rootAdds/BToHNL_MuonAndHNLGenFilter_mHNL1p0_ctau1000.txt') # for when working over jupyter labs
#fileset['phi1000'] = rootAdds('notebooks/Bplus/rootAdds/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000.txt')
#fileset['phi300'] = rootAdds('notebooks/Bplus/rootAdds/BToKPhi_MuonGenFilter_mPhi1p0_ctau300.txt')
store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root


out = processor.run_uproot_job(
    fileset,
    treename="ntuples/llp",
    processor_instance=LLP_ntuple_processor(),
    executor=processor.futures_executor,
    executor_args={"schema": BaseSchema, "workers": 16},
    maxchunks = 1
)

Output()

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/coffea/processor/executor.py", line 1350, in automatic_retries
    raise e
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/coffea/processor/executor.py", line 1335, in automatic_retries
    return func(*args, **kwargs)
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/coffea/processor/executor.py", line 1402, in metadata_fetcher
    tree = file[item.treename]
  File "/uscms_data/d3/aaportel/mambaforge/envs/analysis-env/lib/python3.9/site-packages/uproot/reading.py", line 2088, in __getitem__
    step = step[item]
  File "/uscms_data/d3/aaportel/m

KeyInFileError: not found: 'ntuples' (with any cycle number)

    Available keys: 'NEvents;1', 'MuonSystem;2', 'MuonSystem;1', 'acceptance;1', 'acceptance_met;1'

in file root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root

In [ ]:
fileset

{'hnl': ['root://cmsxrootd.fnal.gov//store/user/lpclonglived/displacedJetMuonNtuple/V1p17/MC_Fall18/v3/BToHNL_MuonAndHNLGenFilter_mHNL1p0_ctau1000/Run2_displacedJetMuonNtupler_V1p17_MC_Fall18_batch1_v3/220331_202058/0000/displacedJetMuon_ntupler_100.root',
  'root://cmsxrootd.fnal.gov//store/user/lpclonglived/displacedJetMuonNtuple/V1p17/MC_Fall18/v3/BToHNL_MuonAndHNLGenFilter_mHNL1p0_ctau1000/Run2_displacedJetMuonNtupler_V1p17_MC_Fall18_batch1_v3/220331_202058/0000/displacedJetMuon_ntupler_101.root',
  'root://cmsxrootd.fnal.gov//store/user/lpclonglived/displacedJetMuonNtuple/V1p17/MC_Fall18/v3/BToHNL_MuonAndHNLGenFilter_mHNL1p0_ctau1000/Run2_displacedJetMuonNtupler_V1p17_MC_Fall18_batch1_v3/220331_202058/0000/displacedJetMuon_ntupler_102.root',
  'root://cmsxrootd.fnal.gov//store/user/lpclonglived/displacedJetMuonNtuple/V1p17/MC_Fall18/v3/BToHNL_MuonAndHNLGenFilter_mHNL1p0_ctau1000/Run2_displacedJetMuonNtupler_V1p17_MC_Fall18_batch1_v3/220331_202058/0000/displacedJetMuon_ntupler_103.

In [26]:
#cuts = ['events_with_llp', 'muon_pt_cut', 'muon_eta_cut', 'csc_acc_cut', 'csc_cls_cut', 'timetotal_cut', 'ME1112_veto', 'eta_cut', 'timeSpread_cut']
cuts = ['events_with_llp', 'muon_cut', 'csc_acc_cut', 'csc_cls_cut', 'timetotal_cut', 'ME1112_veto', 'eta_cut', 'timeSpread_cut']
tot = out['hnl']['events_with_llp']
for i in range(len(cuts)-1):
    print(cuts[i] + ':', out['hnl'][cuts[i]])

events_with_llp: 1333462
muon_cut: 878707
csc_acc_cut: 66519
csc_cls_cut: 14731
timetotal_cut: 12518
ME1112_veto: 3573
eta_cut: 3573
